In [1]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from fastai.text.all import *
from fastai import *
from blurr import *
from transformers import *
import transformers
from blurr.text.data.all import *
from blurr.text.modeling.all import *
import bert_score
from bert_score import score

import re
import numpy as np
import pandas as pd
import gzip
import io
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
from fastbook import load_learner
from tqdm import tqdm
from sklearn.model_selection import train_test_split

/opt/conda/envs/pytorch/lib/python3.9/site-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ec2-user/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
#scp -i ~/Desktop/W210/w210keypair.pem ~/Downloads/new-reddit-data-clean.csv ec2-user@ec2-35-91-71-95.us-west-2.compute.amazonaws.com:~/


In [9]:
#!pip install fastbook
#pip install bert_score
#!pip install transformers
#! pip install sentencepiece
#!pip install ohmeow-blurr

In [2]:
train = pd.read_csv('new-reddit-data-clean.csv')
train.head()

,author,subreddit,selftext,title,title_length,invalid_title_length,cosine_sim
0,wizzle_ra_dizzle,MTB,"Bought the stumpjumper evo expert last week, and I’ve been loving it. As the title says though, the most pleasantly surprising thing about it is that the rear hub is silent.\n\nWhy aren’t more hubs designed like this? It’s fantastic. The bike disappears more in the background for me, and I just enjoy the trail even more.",Most pleasantly surprising thing about the Stumpjumper Evo expert… the silent rear hub,13,0,0.499998
1,BhaveshShaha,india,"While we are aware of the surface level impacts that have led to an adverse standing of women in our nation (please don't generalise entire India with the epistemic bubble carved in a small urban segment and conclude that it's not the case). The National Family Health Survey conducted in years from 1992-93 till 2015-16, India has dropped in the patriarchy index from 34 to 31.\n\nI was wondering of the nuances; what are the unseen impacts of patriarchy on women's status in contemporary India that do not surface up immediately when we think of it but have had a vital impact on their status?",The other unseen impacts of patriarchy on women's status in contemporary India,12,0,0.499996
2,Tenn_10,LucidDreaming,"I never share my lucid dream as they are not as interesting as other who are flying but i had this dream awhile ago me waking up 2 times inside my dream, i really thought i fully woke up and even tried to storytell my cousin(who happens to be in my dream) about the first scene(dream)\n\nwoke up fr nervous as usual but this is scarier as i want to wake up from the first scene(dream)\n\nTbh idk if what im experiencing is lucid dream. But it just me having self awareness inside my dream and can somehow move my body, and i often experience this for no reason.",Woke up 2 times in my dream but still inside the dream,12,0,0.499994
3,67yu56utty,onepiecefilmredfree,"Toei Animation Company! Here’s options for downloading or watching One Piece Film: Red streaming the full movie online for free on 123movies &amp; Reddit, 1movies, 9movies, and yes movies, including where to watch the anticipated anime fantasy film at home. Is One Piece Film: Red 2022 available to stream? Is watching One Piece Film: Red on Disney Plus, HBO Max, Netflix or Amazon Prime? Yes we have found an authentic streaming option / service. Details on how you can watch One Piece Film: Red for free throughout the year are described below.\n\n**WATCH HERE ➣➣** [One Piece Film Red STREAMIN...",Where can I watch 'One Piece Film: Red' (2022) free online streaming at home?,14,0,0.499987
4,Zealousideal_Dig9927,honkfree,"This movie is amazing! Thor embarks on a journey unlike anything he's ever faced -- a quest for inner peace. However, his retirement gets interrupted by Gorr the God Butcher, a galactic killer who seeks the extinction of the gods.\n\nDerrickson and Cargill successfully tailor their focused and mostly compelling narrative to a Steven Spielberg/Amblin Entertainment–esque bit of Stephen King–sploitation. (King, as it happens, is author Hill’s dad.) There’s nothing in “Honk for Jesus. Save Your Soul.” that you can’t also get in more inventive recent King adaptations (like “Doctor Sleep”) or Ki...",Watch Honk for Jesus. Save Your Soul. Online Free,9,0,0.499978


In [3]:
len(train)

39989

In [4]:
def proprocessing(sample): #preprocessing
    sample = re.sub(r"http\S+", "", sample) #url
    sample = re.sub('\n','', sample) #/n
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'  #special characters
    sample = re.sub(pat, '', sample)
    return sample

In [5]:
proprocessing(train['selftext'][16])

'Check out [Amazon Gold Box Daily Deals]       Check out the [Top Deals on Sneakers] for Sale on Amazon!       Get An OnFeet Look At The Why So Sad? x Nike SB Dunk Low      Making its debut later on this year, here are official images of the Why So Sad? x [Nike SB Dunk Low] collaboration.Created by John Rattray skateboarder and advocate for mental health awareness, the mission of Why So Sad? is to help open up the conversation about suicide prevention in skateboarding and elsewhere. Helping the cause, this collaboration between [Nike SB] and Scottish talent will be a large part of this years fundraising efforts.Shown below, the Why So Sad? x Nike SB Dunk Low features a Light Blue leather base and mesh tongues paired up with Dark Blue tearaway overlays to reveal Yellow underneath. Other notable details include the embroidered skateboarding chicken motif on the lateral heel, the cobranded insoles with graphic of a bird doing a sad grab, Why So Sad? branding on the inner tongue tag, and q

In [6]:
train['cleantext']= train['selftext'].apply(lambda x:proprocessing(x))

In [7]:
train.head()

,author,subreddit,selftext,title,title_length,invalid_title_length,cosine_sim,cleantext
0,wizzle_ra_dizzle,MTB,"Bought the stumpjumper evo expert last week, and I’ve been loving it. As the title says though, the most pleasantly surprising thing about it is that the rear hub is silent.\n\nWhy aren’t more hubs designed like this? It’s fantastic. The bike disappears more in the background for me, and I just enjoy the trail even more.",Most pleasantly surprising thing about the Stumpjumper Evo expert… the silent rear hub,13,0,0.499998,"Bought the stumpjumper evo expert last week, and Ive been loving it. As the title says though, the most pleasantly surprising thing about it is that the rear hub is silent.Why arent more hubs designed like this? Its fantastic. The bike disappears more in the background for me, and I just enjoy the trail even more."
1,BhaveshShaha,india,"While we are aware of the surface level impacts that have led to an adverse standing of women in our nation (please don't generalise entire India with the epistemic bubble carved in a small urban segment and conclude that it's not the case). The National Family Health Survey conducted in years from 1992-93 till 2015-16, India has dropped in the patriarchy index from 34 to 31.\n\nI was wondering of the nuances; what are the unseen impacts of patriarchy on women's status in contemporary India that do not surface up immediately when we think of it but have had a vital impact on their status?",The other unseen impacts of patriarchy on women's status in contemporary India,12,0,0.499996,"While we are aware of the surface level impacts that have led to an adverse standing of women in our nation please don't generalise entire India with the epistemic bubble carved in a small urban segment and conclude that it's not the case. The National Family Health Survey conducted in years from 199293 till 201516, India has dropped in the patriarchy index from 34 to 31.I was wondering of the nuances; what are the unseen impacts of patriarchy on women's status in contemporary India that do not surface up immediately when we think of it but have had a vital impact on their status?"
2,Tenn_10,LucidDreaming,"I never share my lucid dream as they are not as interesting as other who are flying but i had this dream awhile ago me waking up 2 times inside my dream, i really thought i fully woke up and even tried to storytell my cousin(who happens to be in my dream) about the first scene(dream)\n\nwoke up fr nervous as usual but this is scarier as i want to wake up from the first scene(dream)\n\nTbh idk if what im experiencing is lucid dream. But it just me having self awareness inside my dream and can somehow move my body, and i often experience this for no reason.",Woke up 2 times in my dream but still inside the dream,12,0,0.499994,"I never share my lucid dream as they are not as interesting as other who are flying but i had this dream awhile ago me waking up 2 times inside my dream, i really thought i fully woke up and even tried to storytell my cousinwho happens to be in my dream about the first scenedreamwoke up fr nervous as usual but this is scarier as i want to wake up from the first scenedreamTbh idk if what im experiencing is lucid dream. But it just me having self awareness inside my dream and can somehow move my body, and i often experience this for no reason."
3,67yu56utty,onepiecefilmredfree,"Toei Animation Company! Here’s options for downloading or watching One Piece Film: Red streaming the full movie online for free on 123movies &amp; Reddit, 1movies, 9movies, and yes movies, including where to watch the anticipated anime fantasy film at home. Is One Piece Film: Red 2022 available to stream? Is watching One Piece Film: Red on Disney Plus, HBO Max, Netflix or Amazon Prime? Yes we have found an authentic streaming option / service. Details on how you can watch One Piece Film: Red for free throughout the year are described below.\n\n**WATCH HERE ➣➣** [One Piece Film Red STREAMIN...",W

In [8]:
train_texts = train[['cleantext','title']]

In [9]:
train_texts.head()

,cleantext,title
0,"Bought the stumpjumper evo expert last week, and Ive been loving it. As the title says though, the most pleasantly surprising thing about it is that the rear hub is silent.Why arent more hubs designed like this? Its fantastic. The bike disappears more in the background for me, and I just enjoy the trail even more.",Most pleasantly surprising thing about the Stumpjumper Evo expert… the silent rear hub
1,"While we are aware of the surface level impacts that have led to an adverse standing of women in our nation please don't generalise entire India with the epistemic bubble carved in a small urban segment and conclude that it's not the case. The National Family Health Survey conducted in years from 199293 till 201516, India has dropped in the patriarchy index from 34 to 31.I was wondering of the nuances; what are the unseen impacts of patriarchy on women's status in contemporary India that do not surface up immediately when we think of it but have had a vital impact on their status?",The other unseen impacts of patriarchy on women's status in contemporary India
2,"I never share my lucid dream as they are not as interesting as other who are flying but i had this dream awhile ago me waking up 2 times inside my dream, i really thought i fully woke up and even tried to storytell my cousinwho happens to be in my dream about the first scenedreamwoke up fr nervous as usual but this is scarier as i want to wake up from the first scenedreamTbh idk if what im experiencing is lucid dream. But it just me having self awareness inside my dream and can somehow move my body, and i often experience this for no reason.",Woke up 2 times in my dream but still inside the dream
3,"Toei Animation Company! Heres options for downloading or watching One Piece Film: Red streaming the full movie online for free on 123movies amp; Reddit, 1movies, 9movies, and yes movies, including where to watch the anticipated anime fantasy film at home. Is One Piece Film: Red 2022 available to stream? Is watching One Piece Film: Red on Disney Plus, HBO Max, Netflix or Amazon Prime? Yes we have found an authentic streaming option / service. Details on how you can watch One Piece Film: Red for free throughout the year are described below.WATCH HERE [One Piece Film Red STREAMING ONLINE]One...",Where can I watch 'One Piece Film: Red' (2022) free online streaming at home?
4,"This movie is amazing! Thor embarks on a journey unlike anything he's ever faced a quest for inner peace. However, his retirement gets interrupted by Gorr the God Butcher, a galactic killer who seeks the extinction of the gods.Derrickson and Cargill successfully tailor their focused and mostly compelling narrative to a Steven Spielberg/Amblin Entertainmentesque bit of Stephen Kingsploitation. King, as it happens, is author Hills dad. Theres nothing in Honk for Jesus. Save Your Soul. that you cant also get in more inventive recent King adaptations like Doctor Sleep or Kinglike homages.Do y...",Watch Honk for Jesus. Save Your Soul. Online Free


In [10]:
train_texts, test_texts = train_test_split(train_texts, test_size=0.1)
len(train_texts)

35990

In [11]:
len(test_texts)

3999

$$T5$$

In [12]:
pretrained_model_name = "t5-base"
hf_arch, hf_config, hf_tokenizer, hf_model = get_hf_objects(pretrained_model_name, 
                                                                  model_cls=T5ForConditionalGeneration)

# Create mini-batch and define parameters
hf_batch_tfm = Seq2SeqBatchTokenizeTransform(hf_arch, hf_config, hf_tokenizer, hf_model, 
    task='summarization')

# Simple preprocessing
preprocessor = SummarizationPreprocessor(
    hf_tokenizer,
    text_attr='cleantext',
    target_text_attr='title',
    max_input_tok_length=256,
    max_target_tok_length=130,
    min_summary_char_length=30,
)

preprocessed_train = preprocessor.process_df(train_texts)


# Prepare data for training
blocks = (Seq2SeqTextBlock(batch_tokenize_tfm=hf_batch_tfm), noop)
dblock = DataBlock(blocks=blocks, get_x=ColReader('cleantext'), get_y=ColReader('title'), splitter=RandomSplitter())
dls = dblock.dataloaders(preprocessed_train, bs = 2)

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the

In [13]:
#Define performance metrics
seq2seq_metrics = {
        'rouge': {
            'compute_kwargs': { 'rouge_types': ["rouge1", "rouge2", "rougeL"], 'use_stemmer': True },
            'returns': ["rouge1", "rouge2", "rougeL"]
        },
        'bertscore': {
            'compute_kwargs': { 'lang': 'fr' },
            'returns': ["precision", "recall", "f1"]}}

#Model
model = BaseModelWrapper(hf_model)
learn_cbs = [BaseModelCallback]
fit_cbs = [Seq2SeqMetricsCallback(custom_metrics=seq2seq_metrics)]

#Specify training
learn = Learner(dls, model,
                opt_func=ranger,loss_func=CrossEntropyLossFlat(),
                cbs=learn_cbs,splitter=partial(blurr_seq2seq_splitter, arch=hf_arch)).to_fp16()


In [13]:
#import torch
#torch.cuda.empty_cache()

In [15]:
#Create optimizer with default hyper-parameters
learn.create_opt() 
learn.freeze()

#Training
learn.fit_one_cycle(3, lr_max=3e-5, cbs=fit_cbs)

#Exporting model
learn.save('t5_reddit_summary.pkl')

epoch,train_loss,valid_loss,rouge1,rouge2,rougeL,bertscore_precision,bertscore_recall,bertscore_f1,time
0,nan,2.011662,0.445413,0.289125,0.415627,0.762611,0.712110,0.734775,1:03:31
1,nan,2.002352,0.444815,0.288848,0.414565,0.758106,0.709734,0.731459,1:06:38
2,nan,2.002392,0.444846,0.288854,0.414580,0.758124,0.709754,0.731479,1:08:13


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

...

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the enviro

/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/opt/conda/envs/pytorch/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of 

	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Path('models/t5_reddit_summary.pkl.pth')

In [14]:
learner = learn.load('t5_reddit_summary.pkl')

In [15]:
learner.export('models/t5_reddit_summary_production.pkl')

In [16]:
inf_learn = load_learner('models/t5_reddit_summary_production.pkl')

In [44]:
content = ["Hello David, my name is George and I'm currently a MIDS candidate at UC Berkeley.I'm a big fan of your work and my team of data scientists and developers have created a product I think you’d be interested in. Do you know the hardest part about coming up with an elevator pitch? It’s trying to condense the most important parts into a concise number of words. That same difficulty can be found on Reddit, one of the fastest growing social media sites in the world. We’ve analyzed reddit data over a 10-year period and found that nearly 20% of all posts and articles had ineffective or erroneous titles. So, what’s the solution? Our product is a web-based app that takes a user’s post and generates a title that has been evaluated to accurately summarize the content. We're still in the early stages of development and would appreciate any feedback, would you be interested in meeting with my team to talk more through our product?"]
#content = ["I'm a big fan of your work and my team of data scientists and developers have created a product I think you’d be interested in. Do you know the hardest part about coming up with an elevator pitch? It’s trying to condense the most important parts into a concise number of words."]
outputs = inf_learn.blurr_generate(content, early_stopping=False, num_return_sequences=1, \
                               min_length=5, max_length=20)
for idx, o in enumerate(outputs):
    print(f'=== Prediction {idx+1} ===\n{o}\n')

=== Prediction 1 ===
{'generated_texts': 'How do you generate a title for Reddit?'}



In [21]:
def f_rscore(reference, candidate): #assuming both are lists, Rouge
    rscores = []
    rscorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    for i in range(len(reference)):
        rscore = rscorer.score(reference[i], candidate[i])
        rscores.append(list(rscore.values())[0][2]) # fscore
    return rscores

In [22]:
def f_bscore(reference, candidate): #assuming both are lists, Bertscore
    P, R, F1 = score(candidate, reference, lang="en")
    return F1.tolist()

In [23]:
def make_predictions(test_texts):
    #list(outputs[0].values())
    predictions = []
    for i in tqdm(range(len(test_texts))):
        outputs = inf_learn.blurr_generate(test_texts.iloc[i].cleantext, early_stopping=False, num_return_sequences=1, \
                                   min_length=5, max_length=60)
        predictions.append(list(outputs[0].values())[0])
    return predictions

In [24]:
def avg_rouge(test_texts, predictions):
    score_list = f_rscore(list(test_texts.title), predictions)
    return np.average(score_list)

In [25]:
def avg_bertscore(test_texts, predictions):
    score_list = f_bscore(list(test_texts.title), predictions)
    return np.average(score_list)

In [26]:
test_texts_sub = test_texts.sample(200)

In [27]:
predictions = make_predictions(test_texts_sub)

100%|█████████████████████████████████████████| 200/200 [01:08<00:00,  2.92it/s]


In [28]:
avg_rouge(test_texts_sub, predictions)

0.4452386867549913

In [29]:
avg_bertscore(test_texts_sub, predictions)

0.8709776720404625